In [ ]:
import cv2
import os
import time
from datetime import datetime
from threading import Thread
from pathlib import Path

# === SETTINGS ===
video_output_folder = "snakeTrain2"
image_output_folder = "snakeDataset2"
os.makedirs(video_output_folder, exist_ok=True)
os.makedirs(image_output_folder, exist_ok=True)

fps = 15.0
cooldown_seconds = 2
min_motion_area = 50
min_record_duration = 10  # seconds
frame_size = (1280, 720)


def adjust_brightness(cap):
    hour = datetime.now().hour
    cap.set(cv2.CAP_PROP_BRIGHTNESS, 0.2 if hour >= 22 or hour < 9 else 1.0)
    return "night" if hour >= 22 or hour < 9 else "day"


def draw_timestamp_overlay(frame, brightness_mode, cam_name):
    now = datetime.now()
    datetime_str = now.strftime("%m/%d/%y %H:%M:%S")
    label = "Day" if brightness_mode == "day" else "Night"

    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 0.6
    color = (0, 0, 255)
    thickness = 2

    lines = [cam_name.upper(), datetime_str, label]
    y_offset = 25
    for i, line in enumerate(lines):
        size = cv2.getTextSize(line, font, scale, thickness)[0]
        x = frame.shape[1] - size[0] - 10
        y = (i + 1) * y_offset
        cv2.putText(frame, line, (x, y), font, scale, color, thickness, cv2.LINE_AA)

    return frame


def extract_training_frames(video_path, base_name):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame25 = int(total_frames * 0.25)
    frame75 = int(total_frames * 0.75)

    for percent, frame_id in zip(["25", "75"], [frame25, frame75]):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        success, frame = cap.read()
        if success:
            out_filename = f"{base_name}_{percent}.jpg"
            out_path = os.path.join(image_output_folder, out_filename)
            cv2.imwrite(out_path, frame)
            print(f"✅ Saved training image: {out_filename}")
        else:
            print(f"❌ Failed to extract {percent}% frame from {base_name}")
    cap.release()


def monitor_camera(camera_index, cam_name):
    def init_camera(index):
        cam = cv2.VideoCapture(index)
        cam.set(cv2.CAP_PROP_FRAME_WIDTH, frame_size[0])
        cam.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_size[1])
        cam.set(cv2.CAP_PROP_FPS, fps)
        return cam

    cap = init_camera(camera_index)
    ret, prev_frame = cap.read()
    if not ret:
        print(f"[{cam_name}] ❌ Failed to access camera.")
        return

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    prev_gray = cv2.GaussianBlur(prev_gray, (21, 21), 0)

    motion_active = False
    last_motion_time = None
    motion_start_time = None
    clip_counter = 0
    video_writer = None

    print(f"[{cam_name}] Watching for motion...")

    while True:
        brightness_mode = adjust_brightness(cap)
        ret, frame = cap.read()
        if not ret:
            print(f"[{cam_name}] ❌ Disconnected. Reinitializing...")
            cap.release()
            time.sleep(1)
            cap = init_camera(camera_index)
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)
        delta = cv2.absdiff(prev_gray, gray)
        thresh = cv2.threshold(delta, 25, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.dilate(thresh, None, iterations=2)
        contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        motion_detected = any(cv2.contourArea(c) > min_motion_area for c in contours)
        current_time = time.time()

        frame = draw_timestamp_overlay(frame, brightness_mode, cam_name)

        if motion_detected:
            last_motion_time = current_time
            if not motion_active:
                clip_counter += 1
                filename = os.path.join(video_output_folder, f"{cam_name}_clip_{clip_counter:04d}.avi")
                fourcc = cv2.VideoWriter_fourcc(*'XVID')
                video_writer = cv2.VideoWriter(filename, fourcc, fps, frame_size)
                motion_start_time = current_time
                motion_active = True
                print(f"[{cam_name}] 🔴 Recording started: {filename}")
            video_writer.write(frame)

        elif motion_active:
            elapsed = current_time - motion_start_time
            video_writer.write(frame)

            if elapsed >= min_record_duration and (current_time - last_motion_time > cooldown_seconds):
                video_writer.release()
                print(f"[{cam_name}] ✅ Clip saved ({elapsed:.1f}s)")
                motion_active = False
                video_writer = None

                base_name = f"{cam_name}_clip_{clip_counter:04d}"
                extract_training_frames(filename, base_name)

        prev_gray = gray

        cv2.imshow(f"{cam_name}", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    if video_writer:
        video_writer.release()
    cap.release()
    print(f"[{cam_name}] Camera released.")


# === Start Threads for Both Cameras ===
thread1 = Thread(target=monitor_camera, args=(0, 'CAM 1'))
thread2 = Thread(target=monitor_camera, args=(1, 'CAM 2'))

thread1.start()
thread2.start()

thread1.join()
thread2.join()

cv2.destroyAllWindows()
print("🎉 All done.")

In [ ]:
#TRAIN AND TEST SNOPT

from ultralytics import YOLO
from roboflow import Roboflow

rf = Roboflow(api_key="z8o24Csri2knE8jtx3g8")
project = rf.workspace("gkala").project("snopticon")
version = project.version(1)
dataset = version.download("yolov8")

# Load a pretrained model (e.g., yolov8n.pt for nano, yolov8s.pt for small)
model = YOLO("yolov8n.pt")  # You can also use yolov8s.pt, yolov8m.pt, etc.

# Train
model.train(data=dataset.location + "/data.yaml",epochs=50,imgsz=640,batch=8, val=False)

# Path to your test images (inside the Roboflow dataset folder)
test_images_path = os.path.join(dataset.location, "test", "images")

# Run prediction on the entire test set
results = model.predict(source=test_images_path,save=True,save_txt=True,conf=0.25,imgsz=640)

In [ ]:
#RUN INFERENCE ON VIDEO USING SNOPT

# Load trained model
model = YOLO("runs/detect/train/weights/best.pt")  # or "yolov8n.pt"

# Path to input video
video_path = "snake_clip.avi"
cap = cv2.VideoCapture(video_path)

# Output writer (optional - saves annotated video)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter("output_snake.avi", fourcc, 30.0,
                      (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 inference on the frame
    results = model(frame, imgsz=640, conf=0.25)

    # Visualize predictions on the frame
    annotated_frame = results[0].plot()

    # Show frame with bounding boxes
    cv2.imshow("YOLOv8 Snake Detection", annotated_frame)

    # Save to file
    out.write(annotated_frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()